# Capstone Week 3: Segmenting and Clustering Neighborhoods in Toronto

## Q1. Scrape Canada postal code from Wikipedia and store data in a dataframe

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

# Extract data from Wikipedia page using BeautifulSoup
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text
table_content = []
soup = BeautifulSoup(data, 'html5lib')
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['Postal Code'] = row.text[:4].strip()
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace('/',',')).replace(')',' ')).strip(' ')
        table_content.append(cell)

In [2]:
# Convert data into Dataframe and correct some data
df = pd.DataFrame(table_content)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
print(df.shape)
df.head(12)

(103, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


## Q2. Add lattitude and longitude coordinates to each neighborhood

In [3]:
# The code was removed by Watson Studio for sharing.

In [4]:
# Merge latitude and longitude columns to the DataFrame
df2 = pd.merge(df, df_data_1, on = 'Postal Code')
df2.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Q3. Explore and Cluster the Neighborhoods in Toronto 

### Import libraries and prepare data

In [5]:
import matplotlib.cm as cm
import matplotlib.colors as colors

import json
from pandas.io.json import json_normalize

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

from sklearn.cluster import KMeans

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [9]:
# Filter for rows that contain the string 'Toronto' in the Borough column. 
df_t = df2[df2['Borough'].str.contains("Toronto")] 
df_t.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_t['Borough'].unique()),
        df_t.shape[0]
    )
)

The dataframe has 7 boroughs and 39 neighborhoods.


In [11]:
# Get the latitude and longitude of Toronto. 
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto 43.6534817, -79.3839347.


### Create a Toronto map with neighborhoods superimposed on it.

In [12]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_t['Latitude'], df_t['Longitude'], df_t['Borough'], df_t['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Credentials and Version

In [13]:
CLIENT_ID = 'WGZQ02ISETQMFJO1C0EBROGST3JRXQFQV4W05CD1CVR0MMCQ' 
CLIENT_SECRET = 'BI0EVUJQAEVVKH3P4D3GZBU3RVMLG55URBMOZY3RJNQOOG3I' 
VERSION = '20180605' 
LIMIT = 100 

### Explore the neighborhood called 'The Beaches' (Index 19).

In [14]:
neighborhood_latitude = df_t.loc[19, 'Latitude'] 
neighborhood_longitude = df_t.loc[19, 'Longitude'] 
neighborhood_name = df_t.loc[19, 'Neighborhood'] 

### Get the top 20 veneus in the beaches within a radius of 600 meters

In [15]:
radius = 600
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            neighborhood_latitude, 
            neighborhood_longitude, 
            radius, 
            LIMIT)
results = requests.get(url).json()

In [16]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-17-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,Beaches Bake Shop,Bakery,43.680363,-79.289692
2,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331
3,The Beech Tree,Gastropub,43.680493,-79.288846
4,Ed's Real Scoop,Ice Cream Shop,43.672630,-79.287993


In [18]:
print('{} venues were returned by Foursquare for top venues within 600 m of \'The Beaches\'.'.format(nearby_venues.shape[0]))

21 venues were returned by Foursquare for top venues within 600 m of 'The Beaches'.


### Explore all neighborhoods in Toronto

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names = df_t['Neighborhood'], latitudes = df_t['Latitude'], longitudes = df_t['Longitude'])

toronto_venues.groupby('Neighborhood').count()

Regent Park , Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
The Danforth  East
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Rosedale
Enclave of M5E
St. Jame

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,59,59,59,59,59,59
"Brockton , Parkdale Village , Exhibition Place",23,23,23,23,23,23
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",17,17,17,17,17,17
Central Bay Street,68,68,68,68,68,68
Christie,16,16,16,16,16,16
Church and Wellesley,80,80,80,80,80,80
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34
Davisville North,8,8,8,8,8,8


### Create one hot encoding to explore all neighborhoods in Toronto

In [20]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.016949,0.0,0.0,0.0,0.0
1,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,"CN Tower , King and Spadina , Railway Lands , ...",0.000000,0.058824,0.058824,0.058824,0.117647,0.117647,0.117647,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,Central Bay Street,0.014706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.014706,0.0,0.0,0.0,0.0
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


### Explore top 10 common venues in the neighborhoods

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [64]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Farmers Market,Restaurant,Beer Bar,Cheese Shop,Seafood Restaurant,Clothing Store,Comfort Food Restaurant
1,"Brockton , Parkdale Village , Exhibition Place",Café,Breakfast Spot,Bakery,Coffee Shop,Grocery Store,Intersection,Burrito Place,Stadium,Bar,Furniture / Home Store
2,"CN Tower , King and Spadina , Railway Lands , ...",Airport Lounge,Airport Service,Airport Terminal,Plane,Airport,Boutique,Boat or Ferry,Sculpture Garden,Bar,Coffee Shop
3,Central Bay Street,Coffee Shop,Sandwich Place,Café,Restaurant,Bubble Tea Shop,Italian Restaurant,Salad Place,Middle Eastern Restaurant,Burger Joint,Spa
4,Christie,Grocery Store,Café,Park,Restaurant,Italian Restaurant,Baby Store,Athletics & Sports,Nightclub,Candy Store,Coffee Shop


### Cluster neighborhoods into 3 clusters using K-Means

In [65]:
kclusters = 3
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [66]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df_t
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head() 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Pub,Café,Bakery,Theater,Electronics Store,Distribution Center,Restaurant,Event Space
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Café,Bubble Tea Shop,Bookstore,Italian Restaurant,Lingerie Store
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cosmetics Shop,Clothing Store,Cocktail Bar,Restaurant,Italian Restaurant,Lingerie Store,Farmers Market,Seafood Restaurant
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Yoga Studio,Moroccan Restaurant,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Bakery,Cocktail Bar,Farmers Market,Restaurant,Beer Bar,Cheese Shop,Seafood Restaurant,Clothing Store,Comfort Food Restaurant


### Visualize results using Folium

In [67]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Park,Pub,Café,Bakery,Theater,Electronics Store,Distribution Center,Restaurant,Event Space
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Café,Bubble Tea Shop,Bookstore,Italian Restaurant,Lingerie Store
15,Downtown Toronto,0,Coffee Shop,Café,Cosmetics Shop,Clothing Store,Cocktail Bar,Restaurant,Italian Restaurant,Lingerie Store,Farmers Market,Seafood Restaurant
19,East Toronto,0,Health Food Store,Trail,Pub,Yoga Studio,Moroccan Restaurant,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store
20,Downtown Toronto,0,Coffee Shop,Bakery,Cocktail Bar,Farmers Market,Restaurant,Beer Bar,Cheese Shop,Seafood Restaurant,Clothing Store,Comfort Food Restaurant
24,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Restaurant,Bubble Tea Shop,Italian Restaurant,Salad Place,Middle Eastern Restaurant,Burger Joint,Spa
25,Downtown Toronto,0,Grocery Store,Café,Park,Restaurant,Italian Restaurant,Baby Store,Athletics & Sports,Nightclub,Candy Store,Coffee Shop
30,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Gym,Thai Restaurant,Vegetarian / Vegan Restaurant,Bar,Concert Hall
31,West Toronto,0,Bakery,Pharmacy,Grocery Store,Supermarket,Portuguese Restaurant,Park,Music Venue,Middle Eastern Restaurant,Gym / Fitness Center,Café
36,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Scenic Lookout,Brewery,Restaurant,Fried Chicken Joint,Music Venue,Sporting Goods Shop


#### Cluster 2

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,1,Trail,Yoga Studio,Moroccan Restaurant,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant


#### Cluster 3

In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,East York/East Toronto,2,Park,Convenience Store,Yoga Studio,Moroccan Restaurant,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant
61,Central Toronto,2,Park,Bus Line,Swim School,Yoga Studio,Monument / Landmark,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store
68,Central Toronto,2,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Molecular Gastronomy Restaurant,Lounge,Malay Restaurant,Market,Martial Arts School
91,Downtown Toronto,2,Park,Playground,Trail,Yoga Studio,Monument / Landmark,Lounge,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant


#### Majority of the neighborhoods fall under Cluster 1, where the most common venues are mainly coffee shops and restaurants. Cluster 2 only has 1 neighborhood and for Cluster 3, the most common venues are all park. 

### Rename clusters

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels']== 0, 'Cluster Labels'] = 'Cafe & Restaurant'
toronto_merged.loc[toronto_merged['Cluster Labels']== 1, 'Cluster Labels'] = 'Trial'
toronto_merged.loc[toronto_merged['Cluster Labels']== 2, 'Cluster Labels'] = 'Park'
toronto_merged.reset_index()

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,Cafe & Restaurant,Coffee Shop,Park,Pub,Café,Bakery,Theater,Electronics Store,Distribution Center,Restaurant,Event Space
1,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Cafe & Restaurant,Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Café,Bubble Tea Shop,Bookstore,Italian Restaurant,Lingerie Store
2,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,Cafe & Restaurant,Coffee Shop,Café,Cosmetics Shop,Clothing Store,Cocktail Bar,Restaurant,Italian Restaurant,Lingerie Store,Farmers Market,Seafood Restaurant
3,19,M4E,East Toronto,The Beaches,43.676357,-79.293031,Cafe & Restaurant,Health Food Store,Trail,Pub,Yoga Studio,Moroccan Restaurant,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store
4,20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,Cafe & Restaurant,Coffee Shop,Bakery,Cocktail Bar,Farmers Market,Restaurant,Beer Bar,Cheese Shop,Seafood Restaurant,Clothing Store,Comfort Food Restaurant
5,24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,Cafe & Restaurant,Coffee Shop,Sandwich Place,Café,Restaurant,Bubble Tea Shop,Italian Restaurant,Salad Place,Middle Eastern Restaurant,Burger Joint,Spa
6,25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,Cafe & Restaurant,Grocery Store,Café,Park,Restaurant,Italian Restaurant,Baby Store,Athletics & Sports,Nightclub,Candy Store,Coffee Shop
7,30,M5H,Downtown Toronto,"Richmond , Adelaide , King",43.650571,-79.384568,Cafe & Restaurant,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Gym,Thai Restaurant,Vegetarian / Vegan Restaurant,Bar,Concert Hall
8,31,M6H,West Toronto,"Dufferin , Dovercourt Village",43.669005,-79.442259,Cafe & Restaurant,Bakery,Pharmacy,Grocery Store,Supermarket,Portuguese Restaurant,Park,Music Venue,Middle Eastern Restaurant,Gym / Fitness Center,Café
9,35,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106,Park,Park,Convenience Store,Yoga Studio,Moroccan Restaurant,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant
